In [1]:
import requests 
from bs4 import BeautifulSoup
from urllib.parse import urlencode

In [2]:
import pandas as pd
import numpy as np

In [3]:
## 위도 경도 추출
def get_addr_info(keyword=""):
    url = "https://apis.zigbang.com/search/all?q={}&type=oneroom".format(keyword)
    response = requests.get(url)
    json_obj = response.json()
    return json_obj["items"][0]["lat"], json_obj["items"][0]["lng"]
    
# test code    
lat, lng = get_addr_info("성수동")
lat, lng

(37.541725158691406, 127.04335021972656)

In [4]:
def get_ids(lat, lng):
    
    url = "https://api.zigbang.com/v3/items2?\
lat_south={}&lat_north={}&lng_west={}&lng_east={}\
&room=[01,02,03,04,05]".format(lat - 0.01, lat + 0.01, lng - 0.01, lng + 0.01)
    
    params = {
        "lat_south": lat - 0.01,
        "lat_north": lat + 0.01,
        "lng_west": lng - 0.01,
        "lng_east": lng + 0.01,
        "room": "[01,02,03,04,05]",
    }
    params_str = urlencode(params)
    url = "https://api.zigbang.com/v3/items2?" + params_str
    response = requests.get(url)
    json_obj = response.json()
    items = json_obj["list_items"]
    return [item["simple_item"]["item_id"] for item in items]

# test code
ids = get_ids(lat, lng)
len(ids), ids[:5]

(78, [17450368, 17406601, 17470120, 17468716, 17468916])

In [5]:
from pandas.io.json import json_normalize
def get_items(ids):
    url = "https://api.zigbang.com/v3/items?detail=true&item_ids={}"\
.format(str(ids).replace(" ",""))
    response = requests.get(url)
    json_obj = response.json()
    items = json_obj["items"]
    datas = [item["item"] for item in items]
    result_df = json_normalize(datas)
    filter_colums = ["_floor", "address1", "address2", 
                     "user_email", "rent", "size", "deposit", "options"]
    return result_df[filter_colums]
    
# test code    
result_df = get_items(ids)    
result_df.iloc[:9]

,_floor,address1,address2,user_email,rent,size,deposit,options
0,5층,서울시 성동구 성수동1가,13-442,sang2205@naver.com,50,6.0,1000,"에어컨,냉장고,세탁기,인덕션,전자레인지,신발장,싱크대"
1,6층,서울시 성동구 성수동2가,316-17,bless6141@naver.com,45,5.0,1000,"에어컨,냉장고,세탁기,인덕션,책상,침대,옷장,신발장,싱크대"
2,2층,서울시 성동구 성수동1가,13-338,05highlight05@gmail.com,45,6.5,500,"에어컨,냉장고,세탁기,가스레인지,싱크대"
3,2층,서울시 성동구 성수동1가,13-338,06highlight06@gmail.com,45,6.5,500,"에어컨,냉장고,세탁기,가스레인지,싱크대"
4,2층,서울시 성동구 성수동1가,13-338,25highlight25@gmail.com,45,6.5,500,"에어컨,냉장고,세탁기,가스레인지,싱크대"
5,반지하,서울시 성동구 성수동1가,685-481,appedix81@hanmail.net,40,7.0,500,"신발장,싱크대"
6,2층,서울시 성동구 성수동1가,656-5,70ktcho@hanmail.net,0,8.0,5500,"에어컨,냉장고,세탁기"
7,2층,서울시 성동구 성수동1가,13-338,15highlight15@gmail.com,45,6.5,500,"에어컨,냉장고,가스레인지,싱크대"
8,1층,서울시 성동구 성수동1가,13-394,powerhuman@naver.com,35,7.0,500,"냉장고,세탁기,가스레인지,책상,침대,싱크대"


In [6]:
def main(addr=""):
    lat, lng = get_addr_info(addr)
    ids = get_ids(lat, lng)
#     print(len(ids))
    return get_items(ids)    

In [7]:
# test code
aa = main('부평구') 
aa

,_floor,address1,address2,user_email,rent,size,deposit,options
0,2층,인천시 부평구 부평동,12-167,ykpjjang@hanmail.net,40,7.0,300,"에어컨,냉장고,세탁기,인덕션,신발장,싱크대"
1,5층,인천시 부평구 부평동,415-30,yhlee2076@naver.com,54,7.0,70,"에어컨,냉장고,세탁기,인덕션,옷장,신발장,싱크대"
2,6층,인천시 부평구 부평동,415-30,sahera@naver.com,52,9.0,70,"에어컨,냉장고,세탁기,인덕션,옷장,신발장,싱크대"
3,2층,인천시 부평구 부평동,10-485,ykpjjang@hanmail.net,45,7.0,50,"에어컨,냉장고,세탁기,인덕션,옷장,신발장,싱크대"
4,2층,인천시 부평구 부평동,909,ykpjjang@hanmail.net,48,7.0,100,"에어컨,냉장고,세탁기,인덕션,책상,옷장,신발장,싱크대"
5,2층,인천시 부평구 부평동,894-8,smsh10054@naver.com,25,8.0,200,"에어컨,냉장고,세탁기,인덕션,신발장,싱크대"
6,4층,인천시 부평구 부평동,431-30,ahrami_@naver.com,48,7.0,60,"에어컨,냉장고,세탁기,인덕션,옷장,신발장,싱크대"
7,2층,인천시 부평구 부평동,10-623,ykpjjang@hanmail.net,45,6.0,100,"에어컨,냉장고,세탁기,인덕션,옷장,신발장,싱크대"
8,7층,인천시 부평구 부평동,415-30,yhlee2076@naver.com,50,9.0,70,"에어컨,냉장고,세탁기,인덕션,옷장,신발장,싱크대"
9,5층,인천시 부평구 부평동,415-26,ahrami_@naver.com,45,8.0,300,"에어컨,냉장고,세탁기,인덕션,옷장,신발장,싱크대"


In [8]:
## json 구조 접근법
json_data # 중첩 json 구조로 되어있음. 'name','version','url','data' 의 key값들 중 'data'에 접근해야함

# json_data['data'] # list의 안의 json 구조로 되어있음. 따라서 우선 리스트로 접근하기

# json_data['data'][0] # 다시 json구조 , 다시 key값으로 해야함

# json_data['data'][0]['서울특별시'] # 결국 서울특별시에 해당하는 구들이 list형태로 생김

# res = [] # 구 list를 담기위한 list
# res.append(json_data['data'][0]['서울특별시']) # res에 리스트 append
# res.append(json_data['data'][1]['부산광역시'])
# res.append(json_data['data'][2]['인천광역시'])

# sum_res = sum(res,[]) #리스트안의 리스트들 구조로되어있기 때문에 접근을 더 쉽게 하기위해 list를 일원화 시킴 [[]] => [] 

NameError: name 'json_data' is not defined

In [9]:
import json
## encoding utf-8
with open('korea-administrative-district.json', encoding='utf-8') as jsonfile: # korea-administrative-district.json 행정구역 json
    json_data = json.load(jsonfile)
## 시 추출    
parsing_list = []
for i in range(0,3):
    parsing_list.append(list(json_data['data'][i].keys())) 

parsing_data = []
# parsing_data.append(list(json_data['data'][0]['서울특별시']))
parsing_data.append(list(json_data['data'][1]['부산광역시']))
# parsing_data.append(list(json_data['data'][2]['인천광역시']))
    
## 구/군 추출
parsing_data = sum(parsing_data,[])

FileNotFoundError: [Errno 2] No such file or directory: 'korea-administrative-district.json'

In [9]:
json_data

{'name': 'korea-administrative-district',
 'version': '20160125',
 'url': 'https://github.com/cosmosfarm/korea-administrative-district',
 'data': [{'서울특별시': ['종로구',
    '중구',
    '용산구',
    '성동구',
    '광진구',
    '동대문구',
    '중랑구',
    '성북구',
    '강북구',
    '도봉구',
    '노원구',
    '은평구',
    '서대문구',
    '마포구',
    '양천구',
    '강서구',
    '구로구',
    '금천구',
    '영등포구',
    '동작구',
    '관악구',
    '서초구',
    '강남구',
    '송파구',
    '강동구']},
  {'부산광역시': ['중구',
    '서구',
    '동구',
    '영도구',
    '부산진구',
    '동래구',
    '남구',
    '북구',
    '강서구',
    '해운대구',
    '사하구',
    '금정구',
    '연제구',
    '수영구',
    '사상구',
    '기장군']},
  {'인천광역시': ['중구',
    '동구',
    '남구',
    '연수구',
    '남동구',
    '부평구',
    '계양구',
    '서구',
    '강화군',
    '옹진군']},
  {'대구광역시': ['중구', '동구', '서구', '남구', '북구', '수성구', '달서구', '달성군']},
  {'광주광역시': ['동구', '서구', '남구', '북구', '광산구']},
  {'대전광역시': ['동구', '중구', '서구', '유성구', '대덕구']},
  {'울산광역시': ['중구', '남구', '동구', '북구', '울주군']},
  {'세종특별자치시': []},
  {'경기도': ['가평군',
    '고양시',
    '과천시',
  

In [13]:

import cx_Oracle # 파이썬 oracle 연동 모듈 

con = cx_Oracle.connect("KIHYUK/3927@192.168.0.216:1521/orcl") # 연결 설정

for data in parsing_data:
    
    df = main(data)
    rows = list(df.itertuples(index=False,name=None))
#   tuple 형태로 넣어줘야함
#   [(0,a,f)] => 0,a,f 들어감( 컬럼에맞게)
    cursor = con.cursor()
    cursor.executemany("INSERT INTO HOUSE_TE(FLOOR,ADDRESS1,ADDRESS2,USER_EMAIL,RENT,SIZES,DEPOSIT,OPTIONS) VALUES(:1,:2,:3,:4,:5,:6,:7,:8)",rows)
    con.commit()

con.close()